In [ ]:
import numpy as np
from numpy import sin,cos
import matplotlib.pyplot as plt

# Shooting Method for Projectile with Drag



In [ ]:
def g_drag(y):
    g = 9.81
    m = 0.2
    Acw = 0.01
    v = np.hypot(y[2],y[3])
    adrag = Acw /m * v*v

    return np.array ([ y[2],
                       y[3],
                       -adrag * y[2]/v,
                       -adrag * y[3]/v - g ] )

# Throwing the Projectile

The throwing is implented as a RK4 method. The integration will terminate when the projectile hits the ground.

In [ ]:
def throw(y,dt=0.01):

    # A list to keep the trajectory
    tr = []

    while y[1] >= 0:

        # add current pos to track
        tr.append(y)

        k1 = dt * g_drag( y )
        k2 = dt * g_drag( y + 0.5*k1 )
        k3 = dt * g_drag( y + 0.5*k2 )
        k4 = dt * g_drag( y + 1.0*k1 )

        y = y + 1./6. * ( k1 + 2*k2 + 2*k3 + k4)


    # linear interpolation with x-axis to find better estimate for last point
    yl = tr[-1]
    tr.append( (yl * y[1] - y * yl[1]) / (y[1] - yl[1]) )

    return np.array(tr)


In [ ]:
angle = 25
velocity = 30

track = throw([0,0,velocity*cos(angle/180*np.pi),velocity*sin(angle/180*np.pi)])
dist = track[-1,0]

print ("distance:", dist)

plt.plot( track[::,0], track[::,1] );

In [ ]:
aim = 50
dist = 0

trials = []

while aim - dist > 0.01:

    if len(trials) == 0:
        # first throw, just a guess
        ty = np.array( [0.0, 0.0, 20.0/np.sqrt(2), 20.0/np.sqrt(2)] )
    elif len(trials) == 1:
        # second throw, try with linear approximation
        ty = aim / trials[0][1] * trials[0][0]
    else:
        # subsequent throws are done via secant method
        t1 = trials[-1][0]
        d1 = trials[-1][1] - aim
        t2 = trials[-2][0]
        d2 = trials[-2][1] - aim

        ty = (t1*d2 - t2*d1) / (d2 - d1)

    track = throw(ty)
    dist = track[-1,0]
    print ( "trial %i: v=%f dist=%f" % (len(trials)+1,
                                        np.hypot(ty[2],ty[3]),
                                        dist) )
    trials.append( [ty,dist] )
    plt.plot( track[::,0], track[::,1] )

# plt.show()